In [0]:
pip install openpyxl pandas

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
dbutils.fs.ls("dbfs:/FileStore/uploads")


Out[5]: [FileInfo(path='dbfs:/FileStore/uploads/Customer.xlsx', name='Customer.xlsx', size=89424, modificationTime=1749473452000),
 FileInfo(path='dbfs:/FileStore/uploads/Orders.json', name='Orders.json', size=3045599, modificationTime=1749473453000),
 FileInfo(path='dbfs:/FileStore/uploads/Products.csv', name='Products.csv', size=173396, modificationTime=1749473451000)]

In [0]:
product_path = '/dbfs/FileStore/uploads/Products.csv'
customer_path = '/dbfs/FileStore/uploads/Customer.xlsx'
order_path = '/dbfs/FileStore/uploads/Orders.json'

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/uploads/"))


path,name,size,modificationTime
dbfs:/FileStore/uploads/Customer.xlsx,Customer.xlsx,89424,1749473452000
dbfs:/FileStore/uploads/Orders.json,Orders.json,3045599,1749473453000
dbfs:/FileStore/uploads/Products.csv,Products.csv,173396,1749473451000


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PEI").getOrCreate()

In [0]:
p_df = pd.read_excel(customer_path, sheet_name="Customer", engine="openpyxl")

#converting to spark df from pandas

customer_df = spark.createDataFrame(p_df)



order_df = df = spark.read.json(order_path)

product_df = spark.read.format("csv") \
    .option("header", "true") \           
    .option("inferSchema", "true") \     
    .option("sep", ",") \                  
    .load(product_path)




In [0]:
#creating raw tables
customer_df.write.format("delta").mode("overwrite").saveAsTable("customer_table")
order_df.write.format("delta").mode("overwrite").saveAsTable("order_table")
product_df.write.format("delta").mode("overwrite").saveAsTable("product_table")


In [0]:
#creating enriched tables for customer and product
enriched_df = customer_df.join(product_df, on="Customer ID", how="inner")
enriched_df = enriched_df.withColumn("profit_rounded", round(col("Profit"), 2))
enriched_df.write.format("delta").mode("overwrite").saveAsTable("enriched_customers_products")


In [0]:
orders_products_df = order_df.join(product_df, "Customer ID", "inner")


In [0]:
#entiched table category and sub category
final_df = enriched_df.select(
    "Order ID",
    "Customer Name",
    "Country",
    "Category",
    "Sub-Category",
    "profit_rounded"
)

final_df.write.format("delta").mode("overwrite").saveAsTable("enriched_orders_customers_products")


In [0]:
#aggregate table
agg_df = orders_products_df.join(customers_df, on="Customer ID", how="left") \
    .withColumn("Order Year", year("Order Date")) \
    .groupBy(
        "Order Year", 
        "Category", 
        "Sub-Category", 
        "Customer Name"
    ) \
    .agg(
        round(sum("Profit"), 2).alias("Total Profit")
    )


agg_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("agg_profit_by_year_category_customer")

In [0]:
%sql
--profit by year
SELECT
  YEAR(`Order Date`) AS Order_Year,
  ROUND(SUM(Profit), 2) AS Total_Profit
FROM order_table
GROUP BY YEAR(`Order Date`)
ORDER BY Order_Year;


In [0]:
%sql
-- profit by year product category
SELECT
  YEAR(o.`Order Date`) AS Order_Year,
  p.`Category`,
  ROUND(SUM(o.Profit), 2) AS Total_Profit
FROM order_table o
JOIN product_table p ON o.`Product ID` = p.`Product ID`
GROUP BY YEAR(o.`Order Date`), p.`Category`
ORDER BY Order_Year, p.`Category`;


In [0]:
%sql
--profit by customer
SELECT
  c.`Customer Name`,
  ROUND(SUM(o.Profit), 2) AS Total_Profit
FROM order_table o
JOIN customer_table c ON o.`Customer ID` = c.`Customer ID`
GROUP BY c.`Customer Name`
ORDER BY Total_Profit DESC;


In [0]:
%sql
--profit by cusotmer year
SELECT
  c.`Customer Name`,
  YEAR(o.`Order Date`) AS Order_Year,
  ROUND(SUM(o.Profit), 2) AS Total_Profit
FROM order_table o
JOIN customer_table c ON o.`Customer ID` = c.`Customer ID`
GROUP BY c.`Customer Name`, YEAR(o.`Order Date`)
ORDER BY c.`Customer Name`, Order_Year;
